Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [3]:
test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,...,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,...,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,...,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [4]:
import numpy as np

def wrangle(X):
  # prevent setting with copy warining
  X = X.copy()

  # About 3% of the time, latitude has small values near zero,
  # outside Tanzania, so we'll treat these values like zero.
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                     'gps_height', 'population']

  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
    X[col+'_MISSING'] = X[col].isnull()

  # Drop duplicates
  duplicates = ['quanity_group', 'payment_type']

  # Drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['id']
  X = X.drop(columns=unusable_variance)

  # convert date recorded to datetime
  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  # extract components from date_time then drop the original column
  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')

  # Engineer feature: how many year from construction_year ro date_recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_MISSING'] = X['years'].isnull()

  return X

In [5]:
train = wrangle(train)
test = wrangle(test)

In [6]:
target = 'status_group'

# get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# get a series with the high cardinality of the numeric numbers
cardinality = train_features.select_dtypes(exclude='number').nunique()

# get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# combine the lists
features = numeric_features + categorical_features

# features = features.drop(columns=[target])

In [7]:
from scipy.stats import randint, uniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

In [16]:
from scipy.stats import randint, uniform

X_train = train[features]
y_train = train[target]

# Mapping the ys to integers for the encoder
mapdict = {
    'functional': 1,
    'non functional': -1,
    'functional needs repair': 0
}
y_train = y_train.map(mapdict)

pipeline = make_pipeline(
    ce.TargetEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor(random_state=42)
)

param_distributions = {
    'targetencoder__min_samples_leaf': randint(1, 1000), 
    'targetencoder__smoothing': uniform(1, 1000), 
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestregressor__n_estimators': randint(50, 500), 
    'randomforestregressor__max_depth': [5, 10, 15, 20, None], 
    'randomforestregressor__max_features': uniform(0, 1), 
}

# If you're on Colab, decrease n_iter & cv parameters
search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 29.2min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 37.8min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 38.2min finished


In [17]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 0.30738304577027475, 'randomforestregressor__n_estimators': 269, 'simpleimputer__strategy': 'mean', 'targetencoder__min_samples_leaf': 934, 'targetencoder__smoothing': 51.75365100651552}
Cross-validation MAE 0.4118270702972025


In [18]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestregressor__max_depth,param_randomforestregressor__max_features,param_randomforestregressor__n_estimators,param_simpleimputer__strategy,param_targetencoder__min_samples_leaf,param_targetencoder__smoothing,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
7,413.957143,7.642926,4.025897,0.150885,None,0.307383,269,mean,934,51.7537,...,-0.410211,-0.413953,-0.411827,0.001570,1,-0.152169,-0.151968,-0.151624,-0.151920,0.000225
5,317.013841,98.872067,7.238197,3.364873,None,0.343945,240,mean,374,546.145,...,-0.456355,-0.453429,-0.454273,0.001481,2,-0.133409,-0.133857,-0.133704,-0.133657,0.000186
3,773.566802,96.258147,5.081713,0.630425,20,0.733642,430,mean,596,199.503,...,-0.476194,-0.470858,-0.472537,0.002588,3,-0.202704,-0.205863,-0.203961,-0.204176,0.001299
8,142.972276,103.850686,6.908338,6.665263,15,0.341166,133,mean,186,69.5072,...,-0.487709,-0.483457,-0.486459,0.002133,4,-0.301315,-0.297554,-0.295835,-0.298235,0.002289
0,73.307782,0.151142,3.157719,0.051236,15,0.160577,324,mean,446,844.294,...,-0.492881,-0.492199,-0.492614,0.000297,5,-0.335912,-0.334780,-0.335578,-0.335423,0.000475
1,122.837383,11.990669,3.425306,0.949129,15,0.411173,210,median,652,792.298,...,-0.498264,-0.496369,-0.497665,0.000917,6,-0.318296,-0.316567,-0.319051,-0.317971,0.001040
6,51.863449,2.944011,2.119139,0.747287,10,0.470321,90,median,73,930.134,...,-0.534190,-0.532932,-0.535078,0.002206,7,-0.393737,-0.393729,-0.394944,-0.394137,0.000571
4,570.089089,23.883697,3.140621,0.129107,10,0.744613,437,median,231,808.316,...,-0.535333,-0.535373,-0.537025,0.002364,8,-0.389526,-0.388370,-0.391108,-0.389668,0.001122
2,375.195781,45.982810,7.549077,8.355724,10,0.964122,186,mean,46,607.39,...,-0.535907,-0.536800,-0.537883,0.002193,9,-0.388078,-0.387470,-0.390370,-0.388639,0.001249
9,53.247565,1.862105,1.248932,0.100357,5,0.886732,123,mean,805,757.925,...,-0.611564,-0.609019,-0.611016,0.001459,10,-0.488218,-0.487958,-0.491674,-0.489284,0.001694


In [20]:
pipeline = search.best_estimator_

In [25]:
from sklearn.metrics import mean_absolute_error

# X_test = test[features]
# y_test = test[target]

y_pred = pipeline.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# print(f'Test MAE: ${mae:,.0f}')

In [26]:
# Unmapping the prediction
y_pred = pd.Series(y_pred)
unmapdict = {value: key for key, value in mapdict.items()}
y_pred = y_pred.map(unmapdict)

In [27]:
# Formatting submission
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('submission-Amer7.csv', index = False)